In [54]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [55]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [56]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2015-07-01 00:00:00")

print(training_set)

# obtengo los nodos del grafo, esta vez los nodos son los usuarios y no los problemas:
nodes = training_set.user_id.unique()

# creo un diccionario que va a tener a los usuarios como keys y los valores seran los
# problemas que ha hecho ese usuario
grouped = training_set.groupby('user_id')['problem_id'].apply(list)

# muestra el numero de usuarios
print(len(nodes))

# muestra la lista de problemas que ha hecho cada usuario
print(grouped)

      problem_id  user_id status       submissionDate
0             10        5     AC  2014-02-17 15:27:07
1              2        6     AC  2014-02-17 15:39:17
2              2        9     AC  2014-02-18 00:30:14
3             10        9     AC  2014-02-18 00:34:46
4              4        9     AC  2014-02-18 00:50:28
5              6        9     AC  2014-02-18 00:52:11
6             13        9     AC  2014-02-18 00:53:40
9             15        8     AC  2014-02-19 19:58:03
10             4        8     AC  2014-02-20 14:23:30
11            39       16     AC  2014-02-20 15:44:33
12            39       17     AC  2014-02-20 16:53:34
13            13       12     AC  2014-02-21 11:08:38
15            33       12     AC  2014-02-21 11:58:32
16            39       12     AC  2014-02-21 12:05:49
17            44       12     AC  2014-02-21 12:52:44
18            44       15     AC  2014-02-21 14:51:52
19            44       22     AC  2014-02-21 14:58:55
21            19        8   

In [57]:
print(training_set)

      problem_id  user_id status       submissionDate
0             10        5     AC  2014-02-17 15:27:07
1              2        6     AC  2014-02-17 15:39:17
2              2        9     AC  2014-02-18 00:30:14
3             10        9     AC  2014-02-18 00:34:46
4              4        9     AC  2014-02-18 00:50:28
5              6        9     AC  2014-02-18 00:52:11
6             13        9     AC  2014-02-18 00:53:40
9             15        8     AC  2014-02-19 19:58:03
10             4        8     AC  2014-02-20 14:23:30
11            39       16     AC  2014-02-20 15:44:33
12            39       17     AC  2014-02-20 16:53:34
13            13       12     AC  2014-02-21 11:08:38
15            33       12     AC  2014-02-21 11:58:32
16            39       12     AC  2014-02-21 12:05:49
17            44       12     AC  2014-02-21 12:52:44
18            44       15     AC  2014-02-21 14:51:52
19            44       22     AC  2014-02-21 14:58:55
21            19        8   

In [58]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2015-07-01 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
6933          117      725     WA  2015-07-01 09:17:38
6934          122      725    RTE  2015-07-01 11:09:50
6935            2     1665     CE  2015-07-01 13:15:06
6936          340     1504     WA  2015-07-01 16:12:24
6937          443     1735     TL  2015-07-01 18:55:30
6938           39     1737     AC  2015-07-01 20:36:18
6939          134     1317     AC  2015-07-02 13:37:18
6940          150     1504     AC  2015-07-02 19:43:11
6941          183     1504     AC  2015-07-02 20:37:12
6942           39     1736     AC  2015-07-03 00:00:53
6943          195     1504     AC  2015-07-03 03:53:16
6944          508     1504     AC  2015-07-03 04:19:22
6945            2     1504     AC  2015-07-03 04:29:37
6946          174     1504     AC  2015-07-03 17:16:06
6947           35      103     AC  2015-07-03 18:10:52
6948          181     1504     AC  2015-07-03 22:08:51
6949            8     1736     AC  2015-07-04 15:46:08
6950      

In [59]:
# In[3]:

def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de usuarios
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de problemas que ha hecho cada usuario
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado


# In[4]:

def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    


# In[5]:

def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo


In [60]:
# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

3431


In [61]:
print(len(nodes))
print(len(links))

912
112850


In [62]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    

In [63]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped.index, 'list_problem_id':grouped.values})

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)

      user_id  len_training  len_evaluation
0           5             3               2
1           6             1               1
2           9             9               5
3           8            40               8
4          16             1               0
5          17             2               2
6          12            12               0
7          18             0              13
8          15             2               2
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            40               2
16         33             7               2
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [64]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))

      user_id  len_training  len_evaluation
2           9             9               5
3           8            40               8
29         60             6              27
118        25             7              33
261       130            62              34
274       382             5               6
285       206           111              38
286        62           112              32
299       414             9             103
310       437           134              13
316       443            41              22
317       448            31              33
333       381            13              10
348       103           109              48
383       542             8              13
395       431            11               8
483       689             5              41
597       254            11              12
601       864            14               8
619       912             7              17
621       840            26              19
633       935            88     

In [65]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users_recommend = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users_recommend)

                                       list_problem_id  user_id
3    [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4                  [2, 10, 4, 6, 13, 253, 57, 62, 143]        9
12                 [258, 141, 309, 310, 390, 187, 510]       25
37                            [100, 62, 2, 57, 53, 23]       60
38   [178, 251, 224, 159, 191, 183, 166, 139, 256, ...       62
69   [33, 51, 60, 70, 83, 134, 181, 187, 191, 206, ...      103
71        [10, 191, 228, 441, 437, 438, 436, 109, 122]      105
88   [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
135  [53, 119, 251, 256, 224, 253, 181, 155, 191, 1...      206
162  [259, 141, 250, 254, 70, 255, 195, 159, 252, 3...      254
224  [259, 235, 226, 51, 119, 316, 258, 352, 272, 2...      381
225                          [119, 262, 264, 114, 388]      382
237         [213, 2, 109, 114, 10, 436, 438, 437, 404]      414
245  [39, 259, 258, 203, 251, 60, 49, 195, 6, 159, 70]      431
248  [250, 124, 249, 100, 4, 122, 57, 30

In [66]:

# en df_new tengo los usuarios a los que tengo que hacer recomendaciones

# primero guardo la lista de usuarios
user_list_recomend = df_users_recommend.user_id.unique()

# creo el nuevo dataframe con los resultados 
column_user_recomend = {'user_id': user_list_to_recommend}
dataframe_user_recomend = pd.DataFrame.from_dict(column_user_recomend)

print(dataframe_user_recomend)

    user_id
0         8
1         9
2        25
3        60
4        62
5       103
6       105
7       130
8       206
9       254
10      381
11      382
12      414
13      431
14      437
15      443
16      448
17      542
18      686
19      689
20      840
21      847
22      864
23      912
24      935
25      979
26     1026
27     1037
28     1136
29     1139
30     1190
31     1212
32     1246
33     1317
34     1386
35     1436
36     1456
37     1460
38     1480
39     1485
40     1504
41     1619


In [67]:
def apply_cn(row, graph):
    """
        Funcion que devuelve el numero de vecinos en comun de esos dos nodos
    """
    return len(list(nx.common_neighbors(graph, row['one'], row['two'])))

def create_cn_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de cn para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_cn = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_cn['cn'] = dataFrame_cn.apply (lambda row: apply_cn(row, graph), axis=1)


    return dataFrame_cn

In [68]:
cn_df = create_cn_data(graph, nodes)
print(cn_df)

         one   two  cn
0          5     6   0
1          5     9   0
2          5     8   0
3          5    16   0
4          5    17   0
5          5    12   0
6          5    15   0
7          5    22   0
8          5     7   0
9          5    28   0
10         5    29   0
11         5    32   0
12         5    35   0
13         5    33   0
14         5    31   0
15         5    40   0
16         5    49   0
17         5    42   0
18         5    46   0
19         5    24   0
20         5    53   0
21         5    60   0
22         5    59   0
23         5    66   0
24         5    68   0
25         5    71   0
26         5    69   0
27         5    70   0
28         5    65   0
29         5    73   0
...      ...   ...  ..
415386  1433  1733   0
415387  1433   636   0
415388  1722  1724   0
415389  1722  1619   0
415390  1722  1725   0
415391  1722  1728   0
415392  1722  1593   0
415393  1722  1733   0
415394  1722   636   0
415395  1724  1619   0
415396  1724  1725   0
415397  172

In [69]:
def getCommonNeighbors(row, cn_df):
    """
        Funcion que devuelve la lista de los usuarios de ese usuario que tienen vecinos en comun
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = cn_df[cn_df['one'] == row['user_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['cn'] > 0]
    column_result_two_tmp = cn_df[cn_df['two'] == row['user_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['cn'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_cn = list_one + list_two
    
    # print(list_cn)
    
    if list_cn == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_cn = graph.nodes()
        list_cn.remove(row['user_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_cn

In [70]:
dataframe_user_recomend['neighbors'] = dataframe_user_recomend.apply (lambda row: getCommonNeighbors(row, cn_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_user_recomend)

    user_id                                          neighbors
0         8  [12, 7, 35, 33, 31, 42, 46, 60, 67, 84, 54, 82...
1         9  [8, 12, 7, 35, 33, 31, 42, 46, 60, 67, 84, 54,...
2        25  [418, 437, 440, 443, 448, 445, 302, 454, 130, ...
3        60  [67, 84, 54, 82, 112, 120, 124, 135, 126, 133,...
4        62  [249, 175, 409, 390, 411, 410, 414, 176, 25, 4...
5       103  [488, 511, 527, 535, 537, 545, 536, 539, 551, ...
6       105  [946, 871, 896, 975, 967, 979, 982, 983, 970, ...
7       130  [461, 396, 483, 103, 488, 511, 527, 535, 537, ...
8       206  [62, 249, 175, 409, 390, 411, 410, 414, 176, 2...
9       254  [901, 876, 912, 840, 864, 930, 933, 813, 934, ...
10      381  [1317, 1274, 1330, 1329, 1334, 1314, 1388, 992...
11      382  [325, 397, 206, 62, 249, 175, 409, 410, 414, 1...
12      414  [176, 25, 418, 437, 440, 443, 448, 445, 302, 4...
13      431  [581, 584, 582, 585, 597, 588, 598, 591, 592, ...
14      437  [440, 443, 448, 445, 302, 454, 130, 461, 3

In [71]:
# ahora voy a separar cada user-problema_a_recomendar para hacer la cuenta
# creo un nuevo dataframe que agrupa por el primer problema y tiene su posible recomendacion
df_separation = dataframe_user_recomend.groupby(['user_id']).neighbors.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation.columns = ['user_id', 'neighbors']

print(df_separation)

       user_id  neighbors
0            8         12
1            8          7
2            8         35
3            8         33
4            8         31
5            8         42
6            8         46
7            8         60
8            8         67
9            8         84
10           8         54
11           8         82
12           8        112
13           8        120
14           8        124
15           8        135
16           8        126
17           8        133
18           8        129
19           8         41
20           8        152
21           8        114
22           8        155
23           8        167
24           8        157
25           8        137
26           8        153
27           8        196
28           8        220
29           8        232
...        ...        ...
11092     1619       1148
11093     1619       1436
11094     1619        161
11095     1619       1441
11096     1619       1456
11097     1619       1480
11098     16

In [72]:
def getProblemsFromSimilarUSers(row, df_users, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de problemas que han
        hecho los usuarios similares a ese. Además eliminara los problemas que ya haya hecho el usuario
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['user_id'] == row['user_id']]
    list_problems_user = list(list_problems_users['list_problem_id'])[0]
    
    # aqui saco la lista de problemas que ha hecho el usuario similar
    list_problems_df = df_users[df_users['user_id'] == row['neighbors']]    
    lista_problemas_comprobar = list(list_problems_df['list_problem_id'])[0]
    
    

    # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
    list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
    
    return list_problems

In [73]:
# ahora para cada lista de de usuarios, hacer una lista de los problemas realizados por esos usuarios, 
# que no los haya realizado ya el usuario
df_separation['list_problems'] = df_separation.apply (lambda row: getProblemsFromSimilarUSers(row, df_users, df_users_recommend), axis=1)

print(df_separation)

       user_id  neighbors                                      list_problems
0            8         12             [33, 39, 44, 100, 51, 49, 70, 81, 130]
1            8          7                            [39, 44, 109, 114, 117]
2            8         35  [8, 17, 23, 25, 29, 31, 33, 35, 47, 49, 51, 62...
3            8         33                       [100, 39, 44, 307, 310, 183]
4            8         31                             [39, 49, 51, 109, 183]
5            8         42      [100, 29, 49, 88, 119, 39, 44, 255, 254, 141]
6            8         46                                     [100, 39, 255]
7            8         60                                  [100, 62, 57, 23]
8            8         67                  [70, 44, 23, 57, 62, 39, 183, 33]
9            8         84  [62, 33, 73, 51, 49, 134, 136, 139, 141, 155, ...
10           8         54        [100, 73, 44, 255, 237, 315, 312, 183, 316]
11           8         82                       [100, 35, 70, 128, 122, 312]

In [74]:
def cn_value(one, two, graph):
    """
        Funcion que devuelve el numero de vecinos en comun de esos dos nodos
    """
    return len(list(nx.common_neighbors(graph, one, two)))


In [75]:
def getValueSimilarMetric(row, df_users_recommend, graph):
    """
        Funcion que va a devolver por cada fila el valor de similaridad entre los dos usuarios de esa fila
    """
    
    return cn_value(row['user_id'], row['neighbors'], graph)

In [76]:
# ahora para cada lista de de usuarios, hacer una lista de los problemas realizados por esos usuarios, 
# que no los haya realizado ya el usuario
df_separation['sim_value'] = df_separation.apply (lambda row: getValueSimilarMetric(row, df_users_recommend, graph), axis=1)

print(df_separation)

       user_id  neighbors                                      list_problems  \
0            8         12             [33, 39, 44, 100, 51, 49, 70, 81, 130]   
1            8          7                            [39, 44, 109, 114, 117]   
2            8         35  [8, 17, 23, 25, 29, 31, 33, 35, 47, 49, 51, 62...   
3            8         33                       [100, 39, 44, 307, 310, 183]   
4            8         31                             [39, 49, 51, 109, 183]   
5            8         42      [100, 29, 49, 88, 119, 39, 44, 255, 254, 141]   
6            8         46                                     [100, 39, 255]   
7            8         60                                  [100, 62, 57, 23]   
8            8         67                  [70, 44, 23, 57, 62, 39, 183, 33]   
9            8         84  [62, 33, 73, 51, 49, 134, 136, 139, 141, 155, ...   
10           8         54        [100, 73, 44, 255, 237, 315, 312, 183, 316]   
11           8         82               

In [77]:
# ahora voy a borrar la columna neighbors
del df_separation['neighbors']

In [78]:
df_separation

,user_id,list_problems,sim_value
0,8,"[33, 39, 44, 100, 51, 49, 70, 81, 130]",23
1,8,"[39, 44, 109, 114, 117]",5
2,8,"[8, 17, 23, 25, 29, 31, 33, 35, 47, 49, 51, 62...",42
3,8,"[100, 39, 44, 307, 310, 183]",9
4,8,"[39, 49, 51, 109, 183]",20
5,8,"[100, 29, 49, 88, 119, 39, 44, 255, 254, 141]",54
6,8,"[100, 39, 255]",15
7,8,"[100, 62, 57, 23]",6
8,8,"[70, 44, 23, 57, 62, 39, 183, 33]",24
9,8,"[62, 33, 73, 51, 49, 134, 136, 139, 141, 155, ...",65


In [79]:
# ahora voy a ordenar por el valor de cn agrupando por el usuario
df_separation = df_separation.sort_values(by=['user_id', 'sim_value'], ascending=False)
print(df_separation)

       user_id                                      list_problems  sim_value
10915     1619  [2, 49, 109, 13, 70, 203, 253, 254, 327, 241, ...         15
10921     1619  [33, 49, 109, 253, 254, 150, 134, 162, 2, 258,...         15
10978     1619  [134, 183, 150, 51, 191, 237, 49, 159, 325, 18...         15
10871     1619  [4, 53, 105, 76, 83, 90, 147, 241, 245, 249, 2...         14
10874     1619  [4, 2, 8, 10, 13, 17, 23, 25, 29, 31, 33, 35, ...         14
10881     1619  [2, 62, 4, 33, 73, 51, 13, 105, 49, 134, 136, ...         14
10884     1619                  [2, 29, 33, 13, 109, 49, 134, 51]         14
10889     1619  [105, 51, 29, 13, 10, 17, 53, 8, 62, 109, 2, 1...         14
10890     1619  [105, 51, 13, 10, 17, 53, 8, 62, 109, 254, 29,...         14
10902     1619  [2, 62, 49, 13, 134, 162, 51, 253, 70, 183, 13...         14
10914     1619  [49, 13, 2, 162, 134, 136, 150, 109, 155, 159,...         14
10919     1619  [53, 119, 251, 256, 224, 253, 181, 155, 191, 1...         14

In [80]:
df_separation[df_separation['user_id'] == 1619]

,user_id,list_problems,sim_value
10915,1619,"[2, 49, 109, 13, 70, 203, 253, 254, 327, 241, ...",15
10921,1619,"[33, 49, 109, 253, 254, 150, 134, 162, 2, 258,...",15
10978,1619,"[134, 183, 150, 51, 191, 237, 49, 159, 325, 18...",15
10871,1619,"[4, 53, 105, 76, 83, 90, 147, 241, 245, 249, 2...",14
10874,1619,"[4, 2, 8, 10, 13, 17, 23, 25, 29, 31, 33, 35, ...",14
10881,1619,"[2, 62, 4, 33, 73, 51, 13, 105, 49, 134, 136, ...",14
10884,1619,"[2, 29, 33, 13, 109, 49, 134, 51]",14
10889,1619,"[105, 51, 29, 13, 10, 17, 53, 8, 62, 109, 2, 1...",14
10890,1619,"[105, 51, 13, 10, 17, 53, 8, 62, 109, 254, 29,...",14
10902,1619,"[2, 62, 49, 13, 134, 162, 51, 253, 70, 183, 13...",14


In [81]:
# ahora voy a borrar la columna de similitud por que ya no me hace falta
del df_separation['sim_value']

In [82]:
df_separation

,user_id,list_problems
10915,1619,"[2, 49, 109, 13, 70, 203, 253, 254, 327, 241, ..."
10921,1619,"[33, 49, 109, 253, 254, 150, 134, 162, 2, 258,..."
10978,1619,"[134, 183, 150, 51, 191, 237, 49, 159, 325, 18..."
10871,1619,"[4, 53, 105, 76, 83, 90, 147, 241, 245, 249, 2..."
10874,1619,"[4, 2, 8, 10, 13, 17, 23, 25, 29, 31, 33, 35, ..."
10881,1619,"[2, 62, 4, 33, 73, 51, 13, 105, 49, 134, 136, ..."
10884,1619,"[2, 29, 33, 13, 109, 49, 134, 51]"
10889,1619,"[105, 51, 29, 13, 10, 17, 53, 8, 62, 109, 2, 1..."
10890,1619,"[105, 51, 13, 10, 17, 53, 8, 62, 109, 254, 29,..."
10902,1619,"[2, 62, 49, 13, 134, 162, 51, 253, 70, 183, 13..."


In [83]:
# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('user_id')

# hago la agregacion en una lista 
df_recommend_final = grouped_r.aggregate(lambda x:list(x))

print(df_recommend_final)

                                             list_problems
user_id                                                   
8        [[124, 100, 122, 57, 307, 311, 308, 312, 313, ...
9        [[15, 19, 27, 60, 53, 105, 76, 83, 90, 147, 24...
25       [[251, 256, 254, 253, 191, 228, 109, 49, 325, ...
60       [[4, 33, 73, 51, 60, 15, 6, 13, 105, 49, 134, ...
62       [[181, 150, 231, 312, 2], [250, 124, 122, 57, ...
103      [[119, 251, 253, 155, 183, 143, 136, 174, 166,...
105      [[2, 203, 174, 258, 136, 254, 308, 49, 226, 16...
130      [[251, 256, 224, 253, 174, 166, 258, 249, 233,...
206      [[178, 162, 147, 73, 51, 90], [250, 124, 122, ...
254      [[2, 62, 4, 33, 73, 51, 60, 15, 6, 13, 105, 49...
381      [[2, 62, 4, 33, 73, 60, 15, 6, 13, 105, 49, 13...
382      [[162, 256, 258, 254, 257, 259, 13], [49, 51, ...
414      [[150, 203, 256, 309, 228, 310, 307, 258, 174,...
431      [[155, 109, 388, 309, 253, 254, 33, 390, 256, ...
437      [[253, 181, 136, 150, 33, 70, 44, 390, 81, 397.

In [84]:
def concatenateLists(l):
    """
        Funcion auxiliar para concatenar listas que estan dentro de una lista
    """
    size = len(l)
    
    result = list()
    
    for i in range(0, size):
        value = l[i]
        result = result + value
    
    return result

In [85]:
def concatenateListsRecom(row):
    """
        Funcion que crea una lista de la concatenacion de listas
    """
    
    value = concatenateLists(row['list_problems'])
    
    return value

In [86]:
# ahora para cada lista de de usuarios, hacer una lista de los problemas realizados por esos usuarios, 
# que no los haya realizado ya el usuario
df_recommend_final['recommendation'] = df_recommend_final.apply (lambda row: concatenateListsRecom(row), axis=1)

print(df_recommend_final)

                                             list_problems  \
user_id                                                      
8        [[124, 100, 122, 57, 307, 311, 308, 312, 313, ...   
9        [[15, 19, 27, 60, 53, 105, 76, 83, 90, 147, 24...   
25       [[251, 256, 254, 253, 191, 228, 109, 49, 325, ...   
60       [[4, 33, 73, 51, 60, 15, 6, 13, 105, 49, 134, ...   
62       [[181, 150, 231, 312, 2], [250, 124, 122, 57, ...   
103      [[119, 251, 253, 155, 183, 143, 136, 174, 166,...   
105      [[2, 203, 174, 258, 136, 254, 308, 49, 226, 16...   
130      [[251, 256, 224, 253, 174, 166, 258, 249, 233,...   
206      [[178, 162, 147, 73, 51, 90], [250, 124, 122, ...   
254      [[2, 62, 4, 33, 73, 51, 60, 15, 6, 13, 105, 49...   
381      [[2, 62, 4, 33, 73, 60, 15, 6, 13, 105, 49, 13...   
382      [[162, 256, 258, 254, 257, 259, 13], [49, 51, ...   
414      [[150, 203, 256, 309, 228, 310, 307, 258, 174,...   
431      [[155, 109, 388, 309, 253, 254, 33, 390, 256, ...   
437     

In [87]:
# ahora voy a borrar la columna de similitud por que ya no me hace falta
del df_recommend_final['list_problems']

In [88]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['recommendation'] if not (x in conjunto_vacio or function_add(x))]

In [89]:
# ahora voy a eliminar las repeticiones
df_recommend_final['recommendation_unique'] = df_recommend_final.apply(lambda row: delRepetitions(row), axis=1)

print(df_recommend_final)

                                            recommendation  \
user_id                                                      
8        [124, 100, 122, 57, 307, 311, 308, 312, 313, 3...   
9        [15, 19, 27, 60, 53, 105, 76, 83, 90, 147, 241...   
25       [251, 256, 254, 253, 191, 228, 109, 49, 325, 2...   
60       [4, 33, 73, 51, 60, 15, 6, 13, 105, 49, 134, 1...   
62       [181, 150, 231, 312, 2, 250, 124, 122, 57, 311...   
103      [119, 251, 253, 155, 183, 143, 136, 174, 166, ...   
105      [2, 203, 174, 258, 136, 254, 308, 49, 226, 162...   
130      [251, 256, 224, 253, 174, 166, 258, 249, 233, ...   
206      [178, 162, 147, 73, 51, 90, 250, 124, 122, 57,...   
254      [2, 62, 4, 33, 73, 51, 60, 15, 6, 13, 105, 49,...   
381      [2, 62, 4, 33, 73, 60, 15, 6, 13, 105, 49, 134...   
382      [162, 256, 258, 254, 257, 259, 13, 49, 51, 33,...   
414      [150, 203, 256, 309, 228, 310, 307, 258, 174, ...   
431      [155, 109, 388, 309, 253, 254, 33, 390, 256, 3...   
437     

In [90]:
del df_recommend_final['recommendation']

In [91]:
df_recommend_final

,recommendation_unique
user_id,
8,"[124, 100, 122, 57, 307, 311, 308, 312, 313, 3..."
9,"[15, 19, 27, 60, 53, 105, 76, 83, 90, 147, 241..."
25,"[251, 256, 254, 253, 191, 228, 109, 49, 325, 2..."
60,"[4, 33, 73, 51, 60, 15, 6, 13, 105, 49, 134, 1..."
62,"[181, 150, 231, 312, 2, 250, 124, 122, 57, 311..."
103,"[119, 251, 253, 155, 183, 143, 136, 174, 166, ..."
105,"[2, 203, 174, 258, 136, 254, 308, 49, 226, 162..."
130,"[251, 256, 224, 253, 174, 166, 258, 249, 233, ..."
206,"[178, 162, 147, 73, 51, 90, 250, 124, 122, 57,..."


In [92]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    if k == 1:
        value = list()
        value.append(row['recommendation_unique'][:k])
        return value
    else:
        return row['recommendation_unique'][:k]

In [93]:
k = 10
# ahora saco los k mejores problemas para cada usuario
df_recommend_final['k_recommendation'] = df_recommend_final.apply(lambda row: getKrecomFinal(row, k), axis=1)

print(df_recommend_final)

                                     recommendation_unique  \
user_id                                                      
8        [124, 100, 122, 57, 307, 311, 308, 312, 313, 3...   
9        [15, 19, 27, 60, 53, 105, 76, 83, 90, 147, 241...   
25       [251, 256, 254, 253, 191, 228, 109, 49, 325, 2...   
60       [4, 33, 73, 51, 60, 15, 6, 13, 105, 49, 134, 1...   
62       [181, 150, 231, 312, 2, 250, 124, 122, 57, 311...   
103      [119, 251, 253, 155, 183, 143, 136, 174, 166, ...   
105      [2, 203, 174, 258, 136, 254, 308, 49, 226, 162...   
130      [251, 256, 224, 253, 174, 166, 258, 249, 233, ...   
206      [178, 162, 147, 73, 51, 90, 250, 124, 122, 57,...   
254      [2, 62, 4, 33, 73, 51, 60, 15, 6, 13, 105, 49,...   
381      [2, 62, 4, 33, 73, 60, 15, 6, 13, 105, 49, 134...   
382      [162, 256, 258, 254, 257, 259, 13, 49, 51, 33,...   
414      [150, 203, 256, 309, 228, 310, 307, 258, 174, ...   
431      [155, 109, 388, 309, 253, 254, 33, 390, 256, 3...   
437     

In [94]:
del df_recommend_final['recommendation_unique']

In [95]:
df_recommend_final

,k_recommendation
user_id,
8,"[124, 100, 122, 57, 307, 311, 308, 312, 313, 315]"
9,"[15, 19, 27, 60, 53, 105, 76, 83, 90, 147]"
25,"[251, 256, 254, 253, 191, 228, 109, 49, 325, 209]"
60,"[4, 33, 73, 51, 60, 15, 6, 13, 105, 49]"
62,"[181, 150, 231, 312, 2, 250, 124, 122, 57, 311]"
103,"[119, 251, 253, 155, 183, 143, 136, 174, 166, ..."
105,"[2, 203, 174, 258, 136, 254, 308, 49, 226, 162]"
130,"[251, 256, 224, 253, 174, 166, 258, 249, 233, 15]"
206,"[178, 162, 147, 73, 51, 90, 250, 124, 122, 57]"


In [96]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
2              [445, 436, 437, 441, 442, 443, 444, 62]        8
3                             [83, 109, 136, 141, 254]        9
9    [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...       25
16   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...       60
17   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...       62
21   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...      103
22   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...      105
24   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...      130
30   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...      206
34   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...      254
43    [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]      381
44                              [33, 8, 465, 6, 2, 13]      382
46   [239, 62, 187, 206, 313, 316, 504, 465, 191, 2...      414
50             [253, 33, 252, 443, 444, 441, 404, 503]      431
51   [489, 181, 390, 393, 397, 437, 486,

In [97]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()
list_recom_problems = df_recommend_final['k_recommendation'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems, 'recom_problems': list_recom_problems}
metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems  \
0             [445, 436, 437, 441, 442, 443, 444, 62]   
1                            [83, 109, 136, 141, 254]   
2   [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...   
3   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...   
4   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...   
5   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...   
6   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...   
7   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...   
8   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...   
9   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...   
10   [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]   
11                             [33, 8, 465, 6, 2, 13]   
12  [239, 62, 187, 206, 313, 316, 504, 465, 191, 2...   
13            [253, 33, 252, 443, 444, 441, 404, 503]   
14  [489, 181, 390, 393, 397, 437, 486, 511, 404, ...   
15  [325, 312, 253, 70, 313, 310, 397, 404, 316, 3...   
16  [124, 488, 145, 228, 505, 5

In [98]:
def one_hit(row):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [99]:
def mrr(row): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """

    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row['recom_problems'])) and (encontrado == False):
            if row['recom_problems'][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [100]:
def precision(row):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['recom_problems']))

In [101]:
def recall(row):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    num_problems_common = np.intersect1d(row['recom_problems'], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [102]:
def f1(row):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    denominador = row['precision'] + row['recall']
    
    if denominador == 0:
        return 0
    else:
        return (2 * row['precision'] * row['recall']) / denominador

In [103]:
# ahora voy a calcular una metrica para cada usuario
metric_df['one_hit'] = metric_df.apply(lambda row: one_hit(row), axis=1)
metric_df['mrr'] = metric_df.apply(lambda row: mrr(row), axis=1)
metric_df['precision'] = metric_df.apply(lambda row: precision(row), axis=1)
metric_df['recall'] = metric_df.apply(lambda row: recall(row), axis=1)
metric_df['f1'] = metric_df.apply(lambda row: f1(row), axis=1)
print(metric_df)

                                        eval_problems  \
0             [445, 436, 437, 441, 442, 443, 444, 62]   
1                            [83, 109, 136, 141, 254]   
2   [438, 13, 228, 143, 251, 502, 503, 508, 509, 4...   
3   [134, 15, 147, 233, 247, 254, 309, 139, 150, 3...   
4   [2, 60, 247, 17, 128, 114, 308, 86, 65, 130, 9...   
5   [35, 507, 119, 76, 81, 93, 95, 97, 105, 117, 1...   
6   [15, 44, 62, 233, 316, 114, 247, 39, 134, 159,...   
7   [471, 486, 503, 509, 25, 86, 316, 325, 327, 33...   
8   [247, 311, 17, 128, 114, 308, 86, 65, 130, 340...   
9   [438, 443, 33, 256, 203, 6, 309, 2, 342, 436, ...   
10   [510, 441, 438, 325, 237, 183, 315, 327, 358, 2]   
11                             [33, 8, 465, 6, 2, 13]   
12  [239, 62, 187, 206, 313, 316, 504, 465, 191, 2...   
13            [253, 33, 252, 443, 444, 441, 404, 503]   
14  [489, 181, 390, 393, 397, 437, 486, 511, 404, ...   
15  [325, 312, 253, 70, 313, 310, 397, 404, 316, 3...   
16  [124, 488, 145, 228, 505, 5

In [104]:
# calculo la media de las metricas

result_one_hit = metric_df['one_hit'].mean()
result_precision = metric_df['precision'].mean()
result_mrr = metric_df['mrr'].mean()
result_recall = metric_df['recall'].mean()
result_f1 = metric_df['f1'].mean()

print("One hit ----------")
print(result_one_hit)
print("Precision ----------")
print(result_precision)
print("Mrr  ----------")
print(result_mrr)
print("Recall  ----------")
print(result_recall)
print("F1  ----------")
print(result_f1)

One hit ----------
0.8571428571428571
Precision ----------
0.30714285714285705
Mrr  ----------
0.4880196523053667
Recall  ----------
0.1327239608648865
F1  ----------
0.16611146599483884


In [105]:

f = open("C:/hlocal/TFM/nodos_usuarios", 'a')
f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 
f.close()